In [ ]:
import fitz  # PyMuPDF
import pandas as pd
import re

pdf_path = "bradesco-relatorio-ESG-2024.pdf"
keywords = ["GRI 405-1", "diversidade", "igualdade","genêro"]

dados = []

# Abre o PDF
doc = fitz.open(pdf_path)
#O formato extraído é um texto
for i, page in enumerate(doc):
    texto = page.get_text("text") or ""
    texto_lower = texto.lower()

    if any(k.lower() in texto_lower for k in keywords):
        # Extrai percentuais (números seguidos de %)
        matches = re.findall(r"(\d+)%", texto)
        for m in matches:
            dados.append({
                "pagina": i + 1,
                "indicador": "GRI 405-1",
                "valor_pct": int(m)
            })

# Cria DataFrame
df = pd.DataFrame(dados)

# Salva em Excel e CSV
df.to_csv("indicadores_gri405_bradesco_mupdf.csv", index=False)
df.to_excel("indicadores_gri405_bradesco_mupdf.xlsx", index=False)

print(df.head())


In [ ]:
from pdfminer.high_level import extract_text
import pandas as pd
import re

pdf_path = "bradesco-relatorio-ESG-2024.pdf"
keywords = ["GRI 405", "diversidade", "igualdade","genêro"]

# Extrai o texto completo
texto = extract_text(pdf_path)

dados = []

# Divide o texto em páginas
paginas = texto.split("\f")  # '\f' = quebra de página

for i, pagina in enumerate(paginas):
    pagina_lower = pagina.lower()
    if any(k.lower() in pagina_lower for k in keywords):
        matches = re.findall(r"(\d+)%", pagina)
        for m in matches:
            dados.append({
                "pagina": i + 1,
                "indicador": "GRI 405-1",
                "valor_pct": int(m)
            })

# Cria DataFrame
df = pd.DataFrame(dados)

# Salva os resultados
df.to_csv("indicadores_gri405_bradesco_pdfminer.csv", index=False)
df.to_excel("indicadores_gri405_bradesco_pdfminer.xlsx", index=False)

print("✅ Extração concluída. Arquivo salvo como 'indicadores_gri405_bradesco_pdfminer.xlsx'")
print(df.head())


In [ ]:
import pdfplumber
import pandas as pd
import re

# Nome do arquivo PDF baixado
pdf_path = "bradesco-relatorio-ESG-2024.pdf"

# Termos para identificar páginas relevantes
keywords = ["GRI 405", "diversidade", "igualdade"]

dados = []

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        texto = page.extract_text() or ""
        texto_lower = texto.lower()

        # Verifica se mencionou GRI 405 ou palavras-chave no texto
        if any(kw.lower() in texto_lower for kw in keywords):
            # Extrai percentuais do texto corrido
            matches = re.findall(r"(\d+)%", texto)
            for m in matches:
                dados.append({
                    "pagina": i + 1,
                    "indicador": "GRI 405 (texto)",
                    "valor_pct": int(m)
                })

            # Também extrai tabelas se houver
            tabelas = page.extract_tables()
            for tabela in tabelas:
                if len(tabela) < 2:
                    continue
                df_tab = pd.DataFrame(tabela[1:], columns=tabela[0])
                # Considera primeira coluna como indicador e segunda como valor
                for _, row in df_tab.iterrows():
                    dados.append({
                        "pagina": i + 1,
                        "indicador": row[tabela[0][0]],
                        "valor_pct": pd.to_numeric(row[tabela[0][1]].replace("%",""), errors="coerce")
                    })

# Cria DataFrame com os resultados
df = pd.DataFrame(dados).dropna(subset=["valor_pct"]).reset_index(drop=True)

# Salva em CSV e Excel para inspeção
df.to_csv("indicadores_gri405_bradesco.csv", index=False)
df.to_excel("indicadores_gri405_bradesco.xlsx", index=False)

df.head()
